# Calculating metrics

xarray is built on top of [numpy](http://numpy.org), which means it implements many of the numpy operators as native methods, and those it doesn't can still be used on the underlying numpy arrays contained with an `xarray.DataArray` object.

This is intended as a short introduction with a focus on climate data. The [xarray documentation on computation](http://xarray.pydata.org/en/stable/computation.html) is excellent and contains many more examples and much more detail.

In [ ]:
import xarray

In [ ]:
%matplotlib inline

Load a CMIP5 air temperature data set into the variable `ds`, and make a variable, `tas`, which directly references the near surface air temperature `DataArray`

In [ ]:
ds = xarray.open_dataset('http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc')
tas = ds.tas

In [ ]:
tas

It is straightforward to calculate the mean temperature for all locations and times in the data

In [ ]:
tas.mean()

It is possible to specify a dimension along which to compute an operator. For example, to calculate the mean in time for all locations specify the `time` dimension as the dimension along which the mean should be calculated:

In [ ]:
tas.mean(dim='time').plot(size=6)

It is common to calculate a 30-year climatology, which is simple using `sel` and chaining operators

In [ ]:
tas_clim = tas.sel(time=slice('1960-01','1989-12')).mean(dim='time')
tas_clim

In [ ]:
tas_clim.plot(size=6)

The anomaly from the 30 year climatology can now be calculated by subtracting it from the original data. xarray checks which named coordinates match between the two data sets, and *broadcasts* the `tas_clim` data along the missing `time` dimension automatically.

In [ ]:
tas_anom = (tas - tas_clim)

The mean global anomaly in time can be calculated by specifying both `lat` and `lon` dimensions in the `dim` argument to `mean`

In [ ]:
tas_anom.sel(time=slice('1960-01',None)).mean(dim=('lat','lon')).plot(size=6)

Note the above plot made use of an open-ended `slice` by specifying `None` as the second argument, which places no upper limit on the `slice` operation.